In [ ]:
# NOT IN USE
# EARLY ATTEMPT PRIOR TO DB CONNECTION AND CHANGE IN DATASET FOR PROJECT

In [ ]:
# install
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

 * Starting PostgreSQL 10 database server
   ...done.
CREATE ROLE


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


'Connected: @postgres'

In [ ]:
# Testing connection
df = %sql SELECT * FROM pg_catalog.pg_tables
df

In [ ]:
#access data from local to Google colab
from google.colab import files
uploaded = files.upload()

Saving Stroke_Mortality_Data.csv to Stroke_Mortality_Data.csv


In [ ]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the cstroke mortality data
stroke_df = pd.read_csv("Stroke_Mortality_Data.csv")
stroke_df.head()

,Year,LocationAbbr,LocationDesc,GeographicLevel,DataSource,Class,Topic,Data_Value,Data_Value_Unit,Data_Value_Type,Data_Value_Footnote_Symbol,Data_Value_Footnote,StratificationCategory1,Stratification1,StratificationCategory2,Stratification2,TopicID,LocationID,Y_lat,X_lon
0,2018,AK,Aleutians East,County,NVSS,Cardiovascular Diseases,Stroke Mortality,2.0,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T6,2013,55.442393,-161.959930
1,2018,AK,Aleutians West,County,NVSS,Cardiovascular Diseases,Stroke Mortality,0.0,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T6,2016,53.648343,-166.917533
2,2018,AK,Anchorage,County,NVSS,Cardiovascular Diseases,Stroke Mortality,63.7,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T6,2020,61.159145,-149.104559
3,2018,AK,Bethel,County,NVSS,Cardiovascular Diseases,Stroke Mortality,105.5,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T6,2050,60.923648,-159.753262
4,2018,AK,Bristol Bay,County,NVSS,Cardiovascular Diseases,Stroke Mortality,NaN,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",~,Insufficient Data,Gender,Overall,Race/Ethnicity,Overall,T6,2060,58.753496,-156.694662


In [ ]:
# Drop the non-beneficial columns
stroke_df = stroke_df.drop(["Year", "DataSource", "Class", "Topic", "Data_Value_Footnote_Symbol", "Data_Value_Footnote", "StratificationCategory1", "StratificationCategory2", "TopicID", "LocationID", "Data_Value_Type", "Data_Value_Unit", "GeographicLevel"], 1)
stroke_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,LocationAbbr,LocationDesc,Data_Value,Stratification1,Stratification2,Y_lat,X_lon
0,AK,Aleutians East,2.0,Overall,Overall,55.442393,-161.959930
1,AK,Aleutians West,0.0,Overall,Overall,53.648343,-166.917533
2,AK,Anchorage,63.7,Overall,Overall,61.159145,-149.104559
3,AK,Bethel,105.5,Overall,Overall,60.923648,-159.753262
4,AK,Bristol Bay,NaN,Overall,Overall,58.753496,-156.694662


In [ ]:
# Rename all columns for clarity
stroke_df.columns =['State', 'Description_Detail', 'Num_Per_100k', 'Gender', 'Ethnicity', 'Latitude_y', 'Longitude_x']

In [ ]:
# Check number of unique values in each column.
stroke_df.nunique()

State                   57
Description_Detail    2022
Num_Per_100k          1696
Gender                   3
Ethnicity                6
Latitude_y            3282
Longitude_x           3282
dtype: int64

In [ ]:
# Check for null values
stroke_df.isnull().sum()

State                     0
Description_Detail        0
Num_Per_100k          30854
Gender                    0
Ethnicity                 0
Latitude_y               18
Longitude_x              18
dtype: int64

In [ ]:
# Drop null values
stroke_df = stroke_df.dropna(axis=0)
stroke_df.head()

,State,Description_Detail,Num_Per_100k,Gender,Ethnicity,Latitude_y,Longitude_x
0,AK,Aleutians East,2.0,Overall,Overall,55.442393,-161.959930
1,AK,Aleutians West,0.0,Overall,Overall,53.648343,-166.917533
2,AK,Anchorage,63.7,Overall,Overall,61.159145,-149.104559
3,AK,Bethel,105.5,Overall,Overall,60.923648,-159.753262
5,AK,Denali,58.0,Overall,Overall,63.678820,-149.960801


In [ ]:
stroke_df.shape

(28222, 7)

In [ ]:
# Check data types to determine if encoding is needed
# Will need to encode State, Description_Detail, Gender, Ethnicity
stroke_df.dtypes

State                  object
Description_Detail     object
Num_Per_100k          float64
Gender                 object
Ethnicity              object
Latitude_y            float64
Longitude_x           float64
dtype: object

In [ ]:
# Generate categorical variable lists
stroke_cat = stroke_df.dtypes[stroke_df.dtypes == "object"].index.tolist()
stroke_cat

['State', 'Description_Detail', 'Gender', 'Ethnicity']

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(stroke_df[stroke_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(stroke_cat)
encode_df.head()

,State_AK,State_AL,State_AR,State_AS,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,...,Description_Detail_Ziebach County,Gender_Female,Gender_Male,Gender_Overall,Ethnicity_American Indian and Alaskan Native,Ethnicity_Asian and Pacific Islander,Ethnicity_Black,Ethnicity_Hispanic,Ethnicity_Overall,Ethnicity_White
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Merge one-hot encoded features and drop the originals
stroke_df = stroke_df.merge(encode_df, left_index=True, right_index=True)
stroke_df = stroke_df.drop(stroke_cat, 1)
stroke_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Num_Per_100k,Latitude_y,Longitude_x,State_AK,State_AL,State_AR,State_AS,State_AZ,State_CA,State_CO,...,Description_Detail_Ziebach County,Gender_Female,Gender_Male,Gender_Overall,Ethnicity_American Indian and Alaskan Native,Ethnicity_Asian and Pacific Islander,Ethnicity_Black,Ethnicity_Hispanic,Ethnicity_Overall,Ethnicity_White
0,2.0,55.442393,-161.959930,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,53.648343,-166.917533,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,63.7,61.159145,-149.104559,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,105.5,60.923648,-159.753262,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,58.0,63.678820,-149.960801,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = stroke_df["Num_Per_100k"].values
X = stroke_df.drop("Num_Per_100k", 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Compile, Train, Evaluate
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 66
hidden_nodes_layer2 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 66)                137280    
                                                                 
 dense_1 (Dense)             (None, 3)                 201       
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 137,485
Trainable params: 137,485
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
323/323 [==============================] - 2s 3ms/step - loss: -7134.2773 - accuracy: 1.9354e-04
Epoch 2/50
323/323 [==============================] - 1s 3ms/step - loss: -124597.1875 - accuracy: 0.0000e+00
Epoch 3/50
323/323 [==============================] - 1s 3ms/step - loss: -547694.0625 - accuracy: 0.0000e+00
Epoch 4/50
323/323 [==============================] - 1s 3ms/step - loss: -1418679.1250 - accuracy: 0.0000e+00
Epoch 5/50
323/323 [==============================] - 1s 3ms/step - loss: -2835991.0000 - accuracy: 0.0000e+00
Epoch 6/50
323/323 [==============================] - 1s 3ms/step - loss: -4870333.5000 - accuracy: 0.0000e+00
Epoch 7/50
323/323 [==============================] - 1s 3ms/step - loss: -7577431.0000 - accuracy: 0.0000e+00
Epoch 8/50
323/323 [==============================] - 1s 3ms/step - loss: -10999278.0000 - accuracy: 0.0000e+00
Epoch 9/50
323/323 [==============================] - 1s 3ms/step - loss: -15166309.0000 - accuracy: 0.0000e+00
Epoc

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

108/108 - 0s - loss: -1.1959e+09 - accuracy: 0.0000e+00 - 311ms/epoch - 3ms/step
Loss: -1195853184.0, Accuracy: 0.0
